In [1]:
%matplotlib inline

from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import LinearRegression


import matplotlib.pyplot as plt
import mediapipe as mp
import cv2
import time
import numpy as np
import pandas as pd
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
classes = ['downdog', 'goddess', 'plank', 'tree', 'warrior2']

# Создаем словарь для замены
mapping = {'downdog': 0, 'goddess': 1, 'plank': 2, 'tree': 3, 'warrior2': 4}

# Загрузка и создание обучающего набора
data_train = pd.read_csv("dataset_train_mediapipe_short.csv")
# Заменяем значения
data_train['target'] = data_train['target'].map(mapping)

X = data_train.iloc[:, 2:-1]
Y = data_train['target']

# Загрузка и создание тестового набора
data_test = pd.read_csv("dataset_test_mediapipe_short.csv")
# Заменяем значения
data_test['target'] = data_test['target'].map(mapping)

X_test = data_test.iloc[:, 2:-1]
Y_test = data_test['target']

In [3]:
# Загрузка и создание обучающего набора на углах
data_train = pd.read_csv("dataset_train_mediapipe_angle.csv")
# Заменяем значения
data_train['target'] = data_train['target'].map(mapping)

X_angle = data_train.iloc[:, 2:-1]
Y_angle = data_train['target']

# Загрузка и создание тестового набора на углах
data_test = pd.read_csv("dataset_test_mediapipe_angle.csv")
# Заменяем значения
data_test['target'] = data_test['target'].map(mapping)

X_test_angle = data_test.iloc[:, 2:-1]
Y_test_angle = data_test['target']

In [4]:
# Отбор признаков 
X_new = X[['Nose_x', 'Nose_y', 'LeftShoulder_x', 'LeftShoulder_y', 'RightShoulder_x', 'RightShoulder_y', 'LeftElbow_x', 'LeftElbow_y', 
                        'RightElbow_x', 'RightElbow_y', 'LeftWrist_x', 'LeftWrist_y', 'RightWrist_x', 'RightWrist_y', 
                        'LeftHip_x', 'LeftHip_y', 'RightHip_x', 'RightHip_y', 
                        'LeftKnee_x', 'LeftKnee_y', 'RightKnee_x', 'RightKnee_y', 'LeftAnkle_x', 'LeftAnkle_y', 'RightAnkle_x', 'RightAnkle_y']].copy()
X_test_new = X_test[['Nose_x', 'Nose_y', 'LeftShoulder_x', 'LeftShoulder_y', 'RightShoulder_x', 'RightShoulder_y', 'LeftElbow_x', 'LeftElbow_y', 
                        'RightElbow_x', 'RightElbow_y', 'LeftWrist_x', 'LeftWrist_y', 'RightWrist_x', 'RightWrist_y', 
                        'LeftHip_x', 'LeftHip_y', 'RightHip_x', 'RightHip_y', 
                        'LeftKnee_x', 'LeftKnee_y', 'RightKnee_x', 'RightKnee_y', 'LeftAnkle_x', 'LeftAnkle_y', 'RightAnkle_x', 'RightAnkle_y']].copy()

In [5]:
#Датасеты на YOLO

# Загрузка и создание обучающего набора
data_train = pd.read_csv("dataset_train_yolo.csv")
# Заменяем значения
data_train['target'] = data_train['target'].map(mapping)

X_yolo = data_train.iloc[:, 2:-1]
Y_yolo = data_train['target']

# Загрузка и создание тестового набора
data_test = pd.read_csv("dataset_test_yolo.csv")
# Заменяем значения
data_test['target'] = data_test['target'].map(mapping)

X_test_yolo = data_test.iloc[:, 2:-1]
Y_test_yolo = data_test['target']

In [6]:
# Загрузка и создание обучающего набора на углах
data_train = pd.read_csv("dataset_train_yolo_angle.csv")
# Заменяем значения
data_train['target'] = data_train['target'].map(mapping)

X_angle_yolo = data_train.iloc[:, 2:-1]
Y_angle_yolo = data_train['target']

# Загрузка и создание тестового набора на углах
data_test = pd.read_csv("dataset_test_yolo_angle.csv")
# Заменяем значения
data_test['target'] = data_test['target'].map(mapping)

X_test_angle_yolo = data_test.iloc[:, 2:-1]
Y_test_angle_yolo = data_test['target']

In [7]:
def classifiers(X, y):
    # Разделение данных на обучающую (80%) и валидационную (20%) выборки
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    print(f"Размеры выборок:")
    print(f"Обучающая: {X_train.shape[0]} примеров")
    print(f"Валидационная: {X_val.shape[0]} примеров")
    
    # Создание списка классификаторов
    dict_classifiers = {
        'Логистическая регрессия': LogisticRegression(random_state=42),
        'Метод опорных векторов': SVC(random_state=42),
        'K-ближайших соседей': KNeighborsClassifier(),
        #'Наивный Байес': MultinomialNB(),
        'Дерево решений': DecisionTreeClassifier(random_state=42),
        'Случайный лес': RandomForestClassifier(random_state=42),
        'AdaBoost': AdaBoostClassifier(random_state=42),
        'Градиентный бустинг': GradientBoostingClassifier(random_state=42)
    }
    
    # Обучение и оценка каждого классификатора
    results = {}
    for name, clf in dict_classifiers.items():
        print(f"\n{'='*50}")
        print(f"Обучение {name}...")
        print(clf)
        # Обучение модели
        clf.fit(X_train, y_train)
        
        # Предсказание на обучающих данных
        y_train_pred = clf.predict(X_train)
        train_accuracy = accuracy_score(y_train, y_train_pred)
        
        # Предсказание на валидационных данных
        y_val_pred = clf.predict(X_val)
        val_accuracy = accuracy_score(y_val, y_val_pred)
        
        print(f"\nМетрики качества:")
        print(f"Точность (Accuracy) на обучении: {train_accuracy:.4f}")
        print(f"Точность (Accuracy) на валидации: {val_accuracy:.4f}")
        
        print("\nОтчет о классификации (валидация):")
        print(classification_report(y_val, y_val_pred))
        
        # Вывод вероятностей для каждого класса (если поддерживается)
        if hasattr(clf, 'predict_proba'):
            y_proba = clf.predict_proba(X_val)
            print("\nСредние вероятности классов (валидация):")
            for i in range(len(classes)):
                print(f"Класс {i} ({classes[i]}): {y_proba[:, i].mean():.4f}")
          
        # Для деревьев и лесов выведем важность признаков
        if hasattr(clf, 'feature_importances_'):
            feature_importance = pd.DataFrame({
                'Признак': X.columns,
                'Важность': clf.feature_importances_
            }).sort_values('Важность', ascending=False)
            print("\nТоп-5 важных признаков:")
            print(feature_importance.head())
        
        # Сохранение результатов для сравнения
        results[name] = {
            'Train Accuracy': train_accuracy,
            'Validation Accuracy': val_accuracy
        }
    
    # Вывод сводной таблицы результатов
    print("\nСводная таблица результатов:")
    results_df = pd.DataFrame.from_dict(results, orient='index')
    print(results_df.sort_values('Validation Accuracy', ascending=False))

In [8]:
def optimal_arh(X, y, names_classifiers):
    # Разделение на обучающую и валидационную выборки
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    print("Размеры выборок:")
    print(f"Обучающая выборка: {X_train.shape}")
    print(f"Валидационная выборка: {X_val.shape}")
    
    # Определение пайплайнов и их параметров
    pipelines = {
        'Логистическая регрессия': Pipeline([
            ('scaler', StandardScaler()),
            ('poly', PolynomialFeatures()),
            ('clf', LogisticRegression())
        ]),
        'SVM': Pipeline([
            ('scaler', StandardScaler()),
            ('poly', PolynomialFeatures()),
            ('clf', SVC())
        ]),
        'Случайный лес': Pipeline([
            ('poly', PolynomialFeatures()),
            ('clf', RandomForestClassifier())
        ]),
        'Градиентный бустинг': Pipeline([
            ('poly', PolynomialFeatures()),
            ('clf', GradientBoostingClassifier())
        ]),
        'K-ближайших соседей': Pipeline([
            ('poly', PolynomialFeatures()),
            ('scaler', StandardScaler()),
            ('clf', KNeighborsClassifier())
        ])
    }
    
    # Параметры для GridSearchCV
    param_grids = {
        'Логистическая регрессия': {
            'scaler': [StandardScaler(), MinMaxScaler()],
            'poly__degree': [1, 2, 3],
            'clf__C': [0.1, 1, 10],
            'clf__penalty': ['l1', 'l2'],
            'clf__solver': ['liblinear']
        },
        'SVM': {
            'scaler': [StandardScaler(), MinMaxScaler()],
            'poly__degree': [1, 2],
            'clf__C': [0.1, 1, 10],
            'clf__kernel': ['rbf', 'linear'],
            'clf__gamma': ['scale', 'auto']
        },
        'Случайный лес': {
            'poly__degree': [1, 2],
            'clf__n_estimators': [100, 200],
            'clf__max_depth': [None, 10, 20],
            'clf__min_samples_split': [2, 5],
            'clf__min_samples_leaf': [1, 2]
        },
        'Градиентный бустинг': {
            'poly__degree': [1, 2],
            'clf__n_estimators': [100, 200],
            'clf__learning_rate': [0.01, 0.1],
            'clf__max_depth': [3, 5],
            'clf__subsample': [0.8, 1.0]
        },
        'K-ближайших соседей': {
            'poly__degree': [1, 2],
            'clf__n_neighbors': range(1, 31),
            'clf__weights': ['uniform', 'distance'],
            'clf__metric': ['euclidean', 'manhattan']
        }
    }
    
    # Обучение и оценка каждого пайплайна
    results = {}
    for name, pipeline in pipelines.items():
        if name in names_classifiers:
            print(f"\n{'='*50}")
            print(f"Оптимизация {name}...")
            
            # Создание GridSearchCV
            grid_search = GridSearchCV(
                pipeline,
                param_grids[name],
                cv=5,
                scoring='accuracy',
                n_jobs=-1,
                verbose=1
            )
            
            # Обучение на обучающей выборке
            grid_search.fit(X_train, y_train)
            
            # Оценка на валидационной выборке
            val_score = grid_search.score(X_val, y_val)
            
            # Вывод результатов
            print(f"\nЛучшие параметры для {name}:")
            for param, value in grid_search.best_params_.items():
                print(f"{param}: {value}")
            
            print(f"\nЛучший результат на обучающей выборке: {grid_search.best_score_:.4f}")
            print(f"Результат на валидационной выборке: {val_score:.4f}")
            
            # Сохранение результатов
            results[name] = {
                'best_params': grid_search.best_params_,
                'train_score': grid_search.best_score_,
                'val_score': val_score,
                'cv_results': pd.DataFrame(grid_search.cv_results_)
            }
    
    # Создание сводной таблицы результатов
    summary = pd.DataFrame({
        'Классификатор': list(results.keys()),
        'Точность на обучающей': [results[name]['train_score'] for name in results.keys()],
        'Точность на валидационной': [results[name]['val_score'] for name in results.keys()]
    }).sort_values('Точность на валидационной', ascending=False)
    
    print("\nСводная таблица результатов:")
    print(summary)
    
    # Сохранение результатов в Excel
    with pd.ExcelWriter('grid_search_results.xlsx') as writer:
        summary.to_excel(writer, sheet_name='Сводная таблица', index=False)
        
        for name, result in results.items():
            result['cv_results'].to_excel(writer, sheet_name=f'{name[:30]}', index=False)

In [9]:
# MEDIAPIPE

# Обучение классификаторов на сокращенном наборе mediapipe
classifiers(X_new, Y)


Размеры выборок:
Обучающая: 770 примеров
Валидационная: 193 примеров

Обучение Логистическая регрессия...
LogisticRegression(random_state=42)

Метрики качества:
Точность (Accuracy) на обучении: 0.7883
Точность (Accuracy) на валидации: 0.7979

Отчет о классификации (валидация):
              precision    recall  f1-score   support

           0       0.97      0.93      0.95        41
           1       0.88      0.66      0.75        32
           2       0.71      0.93      0.81        45
           3       0.94      0.57      0.71        28
           4       0.69      0.79      0.73        47

    accuracy                           0.80       193
   macro avg       0.84      0.78      0.79       193
weighted avg       0.82      0.80      0.80       193


Средние вероятности классов (валидация):
Класс 0 (downdog): 0.2083
Класс 1 (goddess): 0.1718
Класс 2 (plank): 0.2582
Класс 3 (tree): 0.1281
Класс 4 (warrior2): 0.2335

Обучение Метод опорных векторов...
SVC(random_state=42)

Метрики

In [11]:
# Выбор оптимальной архитектуры
optimal_arh(X_new, Y, {'SVM', 'Случайный лес', 'Градиентный бустинг'})

Размеры выборок:
Обучающая выборка: (770, 26)
Валидационная выборка: (193, 26)

Оптимизация SVM...
Fitting 5 folds for each of 48 candidates, totalling 240 fits

Лучшие параметры для SVM:
clf__C: 10
clf__gamma: scale
clf__kernel: rbf
poly__degree: 1
scaler: StandardScaler()

Лучший результат на обучающей выборке: 0.8636
Результат на валидационной выборке: 0.8135

Оптимизация Случайный лес...
Fitting 5 folds for each of 48 candidates, totalling 240 fits

Лучшие параметры для Случайный лес:
clf__max_depth: 10
clf__min_samples_leaf: 1
clf__min_samples_split: 2
clf__n_estimators: 200
poly__degree: 1

Лучший результат на обучающей выборке: 0.8416
Результат на валидационной выборке: 0.7927

Оптимизация Градиентный бустинг...
Fitting 5 folds for each of 32 candidates, totalling 160 fits

Лучшие параметры для Градиентный бустинг:
clf__learning_rate: 0.1
clf__max_depth: 5
clf__n_estimators: 200
clf__subsample: 0.8
poly__degree: 1

Лучший результат на обучающей выборке: 0.8649
Результат на валид

In [12]:
# Обучение классификаторов на наборе mediapipe_angle
classifiers(X_angle, Y_angle)

Размеры выборок:
Обучающая: 770 примеров
Валидационная: 193 примеров

Обучение Логистическая регрессия...
LogisticRegression(random_state=42)

Метрики качества:
Точность (Accuracy) на обучении: 0.4468
Точность (Accuracy) на валидации: 0.3886

Отчет о классификации (валидация):
              precision    recall  f1-score   support

           0       0.50      0.54      0.52        41
           1       0.42      0.56      0.48        32
           2       0.26      0.20      0.23        45
           3       0.70      0.25      0.37        28
           4       0.31      0.40      0.35        47

    accuracy                           0.39       193
   macro avg       0.44      0.39      0.39       193
weighted avg       0.41      0.39      0.38       193


Средние вероятности классов (валидация):
Класс 0 (downdog): 0.1946
Класс 1 (goddess): 0.1729
Класс 2 (plank): 0.2351
Класс 3 (tree): 0.1386
Класс 4 (warrior2): 0.2588

Обучение Метод опорных векторов...
SVC(random_state=42)

Метрики

In [13]:
# Выбор оптимальной архитектуры
optimal_arh(X_angle, Y_angle, {'Случайный лес', 'SVM', 'K-ближайших соседей'})

Размеры выборок:
Обучающая выборка: (770, 8)
Валидационная выборка: (193, 8)

Оптимизация SVM...
Fitting 5 folds for each of 48 candidates, totalling 240 fits

Лучшие параметры для SVM:
clf__C: 10
clf__gamma: auto
clf__kernel: rbf
poly__degree: 2
scaler: StandardScaler()

Лучший результат на обучающей выборке: 0.7156
Результат на валидационной выборке: 0.6943

Оптимизация Случайный лес...
Fitting 5 folds for each of 48 candidates, totalling 240 fits

Лучшие параметры для Случайный лес:
clf__max_depth: None
clf__min_samples_leaf: 1
clf__min_samples_split: 5
clf__n_estimators: 200
poly__degree: 2

Лучший результат на обучающей выборке: 0.7390
Результат на валидационной выборке: 0.7306

Оптимизация K-ближайших соседей...
Fitting 5 folds for each of 240 candidates, totalling 1200 fits

Лучшие параметры для K-ближайших соседей:
clf__metric: manhattan
clf__n_neighbors: 3
clf__weights: distance
poly__degree: 1

Лучший результат на обучающей выборке: 0.7273
Результат на валидационной выборке: 

In [14]:
# Обучение классификаторов на наборе mediapipe_short
classifiers(X, Y)

Размеры выборок:
Обучающая: 770 примеров
Валидационная: 193 примеров

Обучение Логистическая регрессия...
LogisticRegression(random_state=42)

Метрики качества:
Точность (Accuracy) на обучении: 0.7948
Точность (Accuracy) на валидации: 0.8135

Отчет о классификации (валидация):
              precision    recall  f1-score   support

           0       1.00      0.93      0.96        41
           1       0.85      0.69      0.76        32
           2       0.70      0.93      0.80        45
           3       0.90      0.64      0.75        28
           4       0.76      0.79      0.77        47

    accuracy                           0.81       193
   macro avg       0.84      0.80      0.81       193
weighted avg       0.83      0.81      0.81       193


Средние вероятности классов (валидация):
Класс 0 (downdog): 0.2095
Класс 1 (goddess): 0.1784
Класс 2 (plank): 0.2576
Класс 3 (tree): 0.1233
Класс 4 (warrior2): 0.2311

Обучение Метод опорных векторов...
SVC(random_state=42)

Метрики

In [15]:
# Выбор оптимальной архитектуры
optimal_arh(X, Y, {'Случайный лес', 'SVM'})

Размеры выборок:
Обучающая выборка: (770, 66)
Валидационная выборка: (193, 66)

Оптимизация SVM...
Fitting 5 folds for each of 48 candidates, totalling 240 fits

Лучшие параметры для SVM:
clf__C: 10
clf__gamma: scale
clf__kernel: rbf
poly__degree: 1
scaler: StandardScaler()

Лучший результат на обучающей выборке: 0.8675
Результат на валидационной выборке: 0.8135

Оптимизация Случайный лес...
Fitting 5 folds for each of 48 candidates, totalling 240 fits

Лучшие параметры для Случайный лес:
clf__max_depth: None
clf__min_samples_leaf: 2
clf__min_samples_split: 2
clf__n_estimators: 100
poly__degree: 2

Лучший результат на обучающей выборке: 0.8468
Результат на валидационной выборке: 0.7876

Сводная таблица результатов:
   Классификатор  Точность на обучающей  Точность на валидационной
0            SVM               0.867532                   0.813472
1  Случайный лес               0.846753                   0.787565


In [16]:
# YOLO

# Обучение классификаторов на наборе на точках
classifiers(X_yolo, Y_yolo)

Размеры выборок:
Обучающая: 864 примеров
Валидационная: 216 примеров

Обучение Логистическая регрессия...
LogisticRegression(random_state=42)

Метрики качества:
Точность (Accuracy) на обучении: 0.7801
Точность (Accuracy) на валидации: 0.7130

Отчет о классификации (валидация):
              precision    recall  f1-score   support

           0       0.80      0.80      0.80        45
           1       0.71      0.56      0.62        36
           2       0.66      0.77      0.71        53
           3       0.66      0.66      0.66        32
           4       0.73      0.72      0.73        50

    accuracy                           0.71       216
   macro avg       0.71      0.70      0.70       216
weighted avg       0.72      0.71      0.71       216


Средние вероятности классов (валидация):
Класс 0 (downdog): 0.2019
Класс 1 (goddess): 0.1758
Класс 2 (plank): 0.2527
Класс 3 (tree): 0.1454
Класс 4 (warrior2): 0.2242

Обучение Метод опорных векторов...
SVC(random_state=42)

Метрики

In [17]:
# Выбор оптимальной архитектуры
optimal_arh(X_yolo, Y_yolo, {'Случайный лес', 'SVM'})

Размеры выборок:
Обучающая выборка: (864, 34)
Валидационная выборка: (216, 34)

Оптимизация SVM...
Fitting 5 folds for each of 48 candidates, totalling 240 fits

Лучшие параметры для SVM:
clf__C: 10
clf__gamma: scale
clf__kernel: rbf
poly__degree: 1
scaler: StandardScaler()

Лучший результат на обучающей выборке: 0.8415
Результат на валидационной выборке: 0.8750

Оптимизация Случайный лес...
Fitting 5 folds for each of 48 candidates, totalling 240 fits

Лучшие параметры для Случайный лес:
clf__max_depth: 20
clf__min_samples_leaf: 1
clf__min_samples_split: 2
clf__n_estimators: 100
poly__degree: 1

Лучший результат на обучающей выборке: 0.8623
Результат на валидационной выборке: 0.8889

Сводная таблица результатов:
   Классификатор  Точность на обучающей  Точность на валидационной
1  Случайный лес               0.862280                   0.888889
0            SVM               0.841477                   0.875000


In [18]:
# Обучение классификаторов на наборе на углах
classifiers(X_angle_yolo, Y_angle_yolo)

Размеры выборок:
Обучающая: 864 примеров
Валидационная: 216 примеров

Обучение Логистическая регрессия...
LogisticRegression(random_state=42)

Метрики качества:
Точность (Accuracy) на обучении: 0.5903
Точность (Accuracy) на валидации: 0.5556

Отчет о классификации (валидация):
              precision    recall  f1-score   support

           0       0.68      0.60      0.64        45
           1       0.65      0.31      0.42        36
           2       0.45      0.77      0.57        53
           3       0.62      0.31      0.42        32
           4       0.61      0.62      0.61        50

    accuracy                           0.56       216
   macro avg       0.60      0.52      0.53       216
weighted avg       0.59      0.56      0.54       216


Средние вероятности классов (валидация):
Класс 0 (downdog): 0.1975
Класс 1 (goddess): 0.1671
Класс 2 (plank): 0.2560
Класс 3 (tree): 0.1501
Класс 4 (warrior2): 0.2292

Обучение Метод опорных векторов...
SVC(random_state=42)

Метрики

In [19]:
# Выбор оптимальной архитектуры
optimal_arh(X_angle_yolo, Y_angle_yolo, {'Случайный лес', 'Градиентный бустинг'})

Размеры выборок:
Обучающая выборка: (864, 8)
Валидационная выборка: (216, 8)

Оптимизация Случайный лес...
Fitting 5 folds for each of 48 candidates, totalling 240 fits

Лучшие параметры для Случайный лес:
clf__max_depth: 20
clf__min_samples_leaf: 1
clf__min_samples_split: 2
clf__n_estimators: 100
poly__degree: 1

Лучший результат на обучающей выборке: 0.8230
Результат на валидационной выборке: 0.8981

Оптимизация Градиентный бустинг...
Fitting 5 folds for each of 32 candidates, totalling 160 fits

Лучшие параметры для Градиентный бустинг:
clf__learning_rate: 0.1
clf__max_depth: 5
clf__n_estimators: 200
clf__subsample: 0.8
poly__degree: 1

Лучший результат на обучающей выборке: 0.8114
Результат на валидационной выборке: 0.9074

Сводная таблица результатов:
         Классификатор  Точность на обучающей  Точность на валидационной
1  Градиентный бустинг               0.811379                   0.907407
0        Случайный лес               0.822953                   0.898148
